# ML Pipeline

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
import warnings
import lightgbm as lgbm
warnings.filterwarnings("ignore")

In [2]:
# import NLP libraries
import re
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chen_bowen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chen_bowen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chen_bowen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# import sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [4]:
# load data from database
engine = create_engine('sqlite:///../models/data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

create a column encoder for Y to avoid re-encoding at training time

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Define url pattern
    url_re = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Detect and replace urls
    detected_urls = re.findall(url_re, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # tokenize sentences
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # save cleaned tokens
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    
    # remove stopwords
    STOPWORDS = list(set(stopwords.words('english')))
    clean_tokens = [token for token in clean_tokens if token not in STOPWORDS]
    
    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
def build_pipeline():
    
    # build NLP pipeline - count words, tf-idf, multiple output classifier
    pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
       ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100, n_jobs = 6)))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=7)
pipeline = build_pipeline()
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vec',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fa0bff14af0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=6)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def build_report(pipeline, X_test, y_test):
    # predict on the X_test
    y_pred = pipeline.predict(X_test)
    
    # build classification report on every column
    performances = []
    for i in range(len(y_test.columns)):
        performances.append([f1_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                             precision_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro'),
                             recall_score(y_test.iloc[:, i].values, y_pred[:, i], average='micro')])
    # build dataframe
    performances = pd.DataFrame(performances, columns=['f1 score', 'precision', 'recall'],
                                index = y_test.columns)   
    return performances

In [9]:
build_report(pipeline, X_test, y_test)

,f1 score,precision,recall
related,0.812328,0.812328,0.812328
request,0.892585,0.892585,0.892585
offer,0.995728,0.995728,0.995728
aid_related,0.769454,0.769454,0.769454
medical_help,0.918981,0.918981,0.918981
medical_products,0.954837,0.954837,0.954837
search_and_rescue,0.974825,0.974825,0.974825
security,0.982606,0.982606,0.982606
military,0.964907,0.964907,0.964907
child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {'clf__estimator__max_features':['sqrt', 0.5],
              'clf__estimator__n_estimators':[50, 100]}

cv = GridSearchCV(estimator=pipeline, param_grid = parameters, cv = 5, n_jobs = 6)
cv.fit(X_train, y_train)

exception calling callback for <Future at 0x7fa06841ecd0 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/chen_bowen/Anaconda/anaconda3/envs/disaster_reponse_app/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 404, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/Users/chen_bowen/Anaconda/anaconda3/envs/disaster_reponse_app/lib/python3.8/multiprocessing/queues.py", line 116, in get
    return _ForkingPickler.loads(res)
AttributeError: 'WordListCorpusReader' object has no attribute '_unload'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/chen_bowen/Anaconda/anaconda3/envs/disaster_reponse_app/lib/python3.8/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/Users/chen_bowen/Anaconda/anaco

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
build_report(cv, X_test, y_test)

In [ ]:
cv.best_params_

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_improved = Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer()),
                                ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
                            ])
pipeline_improved.fit(X_train, y_train)
y_pred_improved = pipeline_improved.predict(X_test)
build_report(pipeline_improved, X_test, y_test)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('rf_model.pkl', 'wb'))

In [ ]:
pickle.dump(pipeline_improved, open('adaboost_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.